In [24]:
import requests
from bs4 import BeautifulSoup
from collections import Counter
from string import punctuation
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk
import re
nltk.download('stopwords')

stop_words = set(stopwords.words('english'))

urls_text_file_path = r"E:\noknok\urls.txt"
test_words_file_path = r"E:\noknok\words.txt"

f_ptr_url = open(urls_text_file_path,"r")
w_ptr_url = open(test_words_file_path,"r")
urls_data = f_ptr_url.readlines()
words = w_ptr_url.readlines()
words_data=[re.sub(r'\n', '', w) for w in words ]
print(words_data)
print(urls_data)


def remove_stop_words(sentence):
    # print("sentence ===",sentence)
    word_tokens = word_tokenize(sentence)
    filtered_sentence = [w for w in word_tokens if not w.lower() in stop_words]
    return filtered_sentence

def parse_output(c_p_obj):
    word_list = []
    for x in c_p_obj:
        key = x
        value = c_p_obj[key]
        if len(key) > 5 and (bool(re.match('^[a-zA-Z0-9]*$', key)) == True):
            word_list.append(key)

    sentence = ' '.join(word for word in word_list)
    filtered_sentence = remove_stop_words(sentence)
    return Counter(filtered_sentence)

def read_and_process_url(url):

    print("URL ====> ",url)
    r = requests.get(url)
    soup = BeautifulSoup(r.content, features="html.parser")

    text_p = (''.join(s.findAll(text=True))for s in soup.findAll('p'))


    c_p_obj = Counter((x.rstrip(punctuation).lower() for y in text_p for x in y.split()))

    c_p = parse_output(c_p_obj)


    # We get the words within divs
    text_div = (''.join(s.findAll(text=True))for s in soup.findAll('div'))
    
    c_div_obj = Counter((x.rstrip(punctuation).lower() for y in text_div for x in y.split()))
    c_div = parse_output(c_div_obj)
    # We sum the two counters and get a list with words count from most to less common
    total = c_div + c_p
    list_most_common_words = total.most_common() 

    print(list_most_common_words)
    return list_most_common_words

def process_output(output_list):
    result_dict = {}
#     for word in words_data:
#         if word in output_list:
#             if word not in result_dict.keys():
#                 result_dict[word] = 1
#             else:
#                 result_dict[word] += 1
#     return result_dict
    

    for common_words in output_list:
        for x in common_words:
            word = x[0]
            value = x[1]

            if word in words_data:
                if word not in result_dict.keys():
                    result_dict[word] = value
                else:
                    result_dict[word] +=  value
    for word in words_data:
        if word not in result_dict:
            result_dict[word]=0
  

    return result_dict


def main():
    output_list = []
    for url in urls_data:
        common_words = read_and_process_url(url)
        output_list.append(common_words)
#         print(output_list)

    results = process_output(output_list)
    print("**********************************************************************************")
    print("Top three words are:")
    results=Counter(results)
    results.most_common()
    for k, v in results.most_common(3):
        print('%s- %i' % (k, v))
        
    print("**********************************************************************************")
    print("Total number of occurance of each word in the list ")
    for k, v in results.most_common():
        print('%s- %i' % (k, v))
main()

[nltk_data] Downloading package stopwords to C:\Users\SHASHI
[nltk_data]     GOWDA\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['science', 'amazing', 'planet', 'animals', 'filllm', 'internet', 'smallest', 'whenever', 'accessibility', 'initially']
['https://www.livescience.com/3945-history-dinosaurs.html\n', 'https://www.livescience.com/archaeology-to-watch-in-2022\n', 'https://www.discovery.com/nature\n', 'https://www.nature.com/articles/485039a']
URL ====>  https://www.livescience.com/3945-history-dinosaurs.html

[('science', 2), ('amazing', 2), ('planet', 2), ('animals', 2), ('history', 2), ('dinosaurs', 2), ('geggel', 2), ('published', 2), ('million', 2), ('period', 2), ('diverse', 2), ('creatures', 2), ('roamed', 2), ('tyrannosaurus', 2), ('triceratops', 2), ('largest', 2), ('dinosaur', 2), ('smallest', 2), ('pterosaurs', 2), ('feathers', 2), ('extinct', 2), ('successful', 2), ('emerged', 2), ('asteroid', 2), ('slammed', 2), ('evolved', 2), ('mostly', 2), ('enormous', 2), ('beasts', 2), ('existed', 2), ('shrank', 2), ('purposes', 2), ('article', 2), ('unless', 2), ('otherwise', 2), ('roughly', 2), ('change

[('science', 2), ('health', 2), ('purchase', 2), ('archaeological', 2), ('published', 2), ('january', 2), ('predictions', 2), ('archaeologists', 2), ('uncover', 2), ('number', 2), ('stories', 2), ('include', 2), ('discoveries', 2), ('golden', 2), ('qumran', 2), ('scrolls', 2), ('nearby', 2), ('humans', 2), ('started', 2), ('building', 2), ('ceremonial', 2), ('countdown', 2), ('archaeology', 2), ('announced', 2), ('discovery', 2), ('called', 2), ('generated', 2), ('headlines', 2), ('around', 2), ('excavated', 2), ('portion', 2), ('according', 2), ('historical', 2), ('records', 2), ('pharaoh', 2), ('amenhotep', 2), ('palaces', 2), ('expect', 2), ('mysteries', 2), ('akhenaten', 2), ('religion', 2), ('worship', 2), ('rather', 2), ('ancient', 2), ('traditional', 2), ('pantheon', 2), ('discovered', 2), ('pandemic', 2), ('omicron', 2), ('variant', 2), ('spreads', 2), ('travel', 2), ('restrictions', 2), ('coming', 2), ('effect', 2), ('governments', 2), ('implementing', 2), ('lockdowns', 2), ('

[('content', 2), ('visiting', 2), ('browser', 2), ('version', 2), ('limited', 2), ('support', 2), ('obtain', 2), ('experience', 2), ('recommend', 2), ('compatibility', 2), ('internet', 2), ('explorer', 2), ('meantime', 2), ('ensure', 2), ('continued', 2), ('displaying', 2), ('without', 2), ('styles', 2), ('javascript', 2), ('advertisement', 2), ('search', 2), ('account', 2), ('nature', 2), ('article', 2), ('science', 2), ('download', 2), ('published', 2), ('birger', 2), ('volume', 2), ('accesses', 2), ('altmetric', 2), ('metrics', 2), ('details', 2), ('geology', 2), ('schmitz', 2), ('revels', 2), ('evolved', 2), ('together', 2), ('billion', 2), ('living', 2), ('geologists', 2), ('western', 2), ('probably', 2), ('biblical', 2), ('created', 2), ('couple', 2), ('centuries', 2), ('thousands', 2), ('devoted', 2), ('meticulous', 2), ('studies', 2), ('analysing', 2), ('detailed', 2), ('record', 2), ('happened', 2), ('planet', 2), ('throughout', 2), ('geologist', 2), ('robert', 2), ('summarize